In [1]:
import numpy as np
import torch
from tqdm import tqdm

# 参数定义
input_len = 90 * 24   # 输入序列长度
output_len = 90 * 24  # 每次预测长度（90天）
target_len = 365 * 24 # 最终预测长度（365天）

# 用于存储预测值
rolling_preds = []

# 初始输入：X_test 中的第一个样本
x = torch.tensor(X_test[0], dtype=torch.float32).unsqueeze(0).to(device)  # shape: (1, 2160, 7)

# 加载训练好的最佳模型（假设 run_1 为最优）
model = BiLSTM_FullOutput(
    input_size=7,
    hidden_size=128,
    output_len=2160,
    num_layers=2,
    dropout=0.3
).to(device)
model.load_state_dict(torch.load("output/models/pred90/run_1.pth"))
model.eval()

# 预测循环
with torch.no_grad():
    while len(rolling_preds) * 24 < target_len:
        pred = model(x)                  # shape: (1, 2160)
        pred_np = pred.squeeze(0).cpu().numpy()
        rolling_preds.append(pred_np)   # append 2160 hourly preds

        # 构建新的输入窗口：往后滑动90天（或更短）
        new_input = x.squeeze(0).cpu().numpy()  # (2160, 7)
        next_input = np.zeros_like(new_input)   # 占位新输入
        next_input[:-output_len] = new_input[output_len:]  # 后移
        # 用预测值填入 Global_active_power（列 0）
        next_input[-output_len:, 0] = pred_np

        # x 重新赋值
        x = torch.tensor(next_input, dtype=torch.float32).unsqueeze(0).to(device)

# 拼接预测
final_preds_hourly = np.concatenate(rolling_preds, axis=0)[:target_len]  # (8760,)

# 聚合为天
final_preds_daily = final_preds_hourly.reshape(-1, 24).mean(axis=1)  # shape: (365,)

# 输出以供后续可视化
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(final_preds_daily, label="Rolling Forecast", linewidth=2)
plt.xlabel("Day")
plt.ylabel("Average Power (kW)")
plt.title("365-Day Rolling Forecast (90→90)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("output/pics/comparison/pred365/rolling_pred_365days.png", dpi=300)
plt.show()

NameError: name 'X_test' is not defined